In [10]:
import dask, concurrent.futures, time, warnings, os, re, pickle
from osgeo import gdal
import requests as r
import panel as pn
pn.extension('echarts')
import param as pm
import pandas as pd
from collections import OrderedDict as odict
import numpy as np
from dask.distributed import LocalCluster, Client
import xarray as xr
import hvplot.pandas
import hvplot.xarray
import warnings
warnings.filterwarnings('ignore')
from urllib.request import urlopen
from xml.etree.ElementTree import parse,fromstring
from affine import Affine
from pandas import to_datetime
import jinja2 as jj2
from rasterio.crs import CRS
from tempfile import NamedTemporaryFile
from datetime import datetime
from netrc import netrc
from subprocess import Popen
from pyproj import Proj
from src.hls_funcs.masks import mask_hls
from src.hls_funcs.predict import pred_cov, pred_bm, pred_bm_se, xr_cdf
from src.hls_funcs.indices import ndvi_func
import cartopy.crs as ccrs
from bokeh.models.formatters import PrintfTickFormatter
from bokeh.models import NumeralTickFormatter
import stackstac
from subprocess import Popen, DEVNULL, STDOUT
from getpass import getpass
from sys import platform

In [11]:
# Create a LUT dict including the HLS product bands mapped to names
lut = {'HLSS30':
       {'B01': 'COASTAL-AEROSOL',
        'B02': 'BLUE', 
        'B03': 'GREEN', 
        'B04': 'RED', 
        'B05': 'RED-EDGE1',
        'B06': 'RED-EDGE2', 
        'B07': 'RED-EDGE3',
        'B08': 'NIR-Broad',
        'B8A': 'NIR1', 
        'B09': 'WATER-VAPOR',
        'B10': 'CIRRUS',
        'B11': 'SWIR1', 
        'B12': 'SWIR2', 
        'Fmask': 'FMASK'},
       'HLSL30': 
       {'B01': 'COASTAL-AEROSOL',
        'B02': 'BLUE', 
        'B03': 'GREEN', 
        'B04': 'RED', 
        'B05': 'NIR1',
        'B06': 'SWIR1',
        'B07': 'SWIR2', 
        'B09': 'CIRRUS', 
        'B10': 'TIR1', 
        'B11': 'TIR2', 
        'Fmask': 'FMASK'}}

# List of all available/acceptable band names
all_bands = ['ALL', 'COASTAL-AEROSOL', 'BLUE', 'GREEN', 'RED', 'RED-EDGE1', 'RED-EDGE2', 'RED-EDGE3', 
             'NIR1', 'SWIR1', 'SWIR2', 'CIRRUS', 'TIR1', 'TIR2', 'WATER-VAPOR', 'FMASK']

needed_bands = ['BLUE', 'GREEN', 'RED', 'NIR1', 'SWIR1', 'SWIR2', 'FMASK']

In [12]:
def NASA_CMR_STAC(hls_data, aws):
    stac = 'https://cmr.earthdata.nasa.gov/stac/' # CMR-STAC API Endpoint
    stac_response = r.get(stac).json()            # Call the STAC API endpoint
    stac_lp = [s for s in stac_response['links'] if 'LP' in s['title']]  # Search for only LP-specific catalogs

    # LPCLOUD is the STAC catalog we will be using and exploring today
    lp_cloud = r.get([s for s in stac_lp if s['title'] == 'LPCLOUD'][0]['href']).json()
    lp_links = lp_cloud['links']
    lp_search = [l['href'] for l in lp_links if l['rel'] == 'search'][0]  # Define the search endpoint
    lim = 100
    search_query = f"{lp_search}?&limit={lim}"    # Add in a limit parameter to retrieve 100 items at a time.
    bbox_num=[-104.79107047,   40.78311181, -104.67687336,   40.87008987]
    bbox = f'{bbox_num[0]},{bbox_num[1]},{bbox_num[2]},{bbox_num[3]}'  # Defined from ROI bounds
    search_query2 = f"{search_query}&bbox={bbox}"                                                  # Add bbox to query
    date_time = hls_data['date_range'][0]+'/'+hls_data['date_range'][1]  # Define start time period / end time period
    search_query3 = f"{search_query2}&datetime={date_time}"  # Add to query that already includes bbox
    collections = r.get(search_query3).json()['features']    
    hls_collections = [c for c in collections if 'HLS' in c['collection']]
    s30_items = [h for h in hls_collections if h['collection'] == 'HLSS30.v1.5']  # Grab HLSS30 collection
    l30_items = [h for h in hls_collections if h['collection'] == 'HLSL30.v1.5']  # Grab HLSL30 collection
    
    if aws:
        for stac in s30_items:
            for band in stac['assets']:
                stac['assets'][band]['href'] = stac['assets'][band]['href'].replace('https://lpdaac.earthdata.nasa.gov/lp-prod-protected', 
                                                                                    '/vsis3/lp-prod-protected')
                stac['assets'][band]['href'] = stac['assets'][band]['href'].replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected', 
                                                                                    '/vsis3/lp-prod-protected')
                
        for stac in l30_items:
            for band in stac['assets']:
                stac['assets'][band]['href'] = stac['assets'][band]['href'].replace('https://lpdaac.earthdata.nasa.gov/lp-prod-protected', 
                                                                                    '/vsis3/lp-prod-protected')
                stac['assets'][band]['href'] = stac['assets'][band]['href'].replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected', 
                                                                                    '/vsis3/lp-prod-protected')
    return {'S30': s30_items,
            'L30': l30_items}

def setup_netrc(creds,aws):
    urs = 'urs.earthdata.nasa.gov' 
    try:
        netrcDir = os.path.expanduser("~/.netrc")
        netrc(netrcDir).authenticators(urs)[0]
        del netrcDir

    # Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
    except FileNotFoundError:
        homeDir = os.path.expanduser("~")
        Popen('touch {0}.netrc | chmod og-rw {0}.netrc | echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
        Popen('echo login {} >> {}.netrc'.format(creds[0], homeDir + os.sep), shell=True)
        Popen('echo password {} >> {}.netrc'.format(creds[1], homeDir + os.sep), shell=True)
        del homeDir

    # Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
    except TypeError:
        homeDir = os.path.expanduser("~")
        Popen('echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
        Popen('echo login {} >> {}.netrc'.format(creds[0], homeDir + os.sep), shell=True)
        Popen('echo password {} >> {}.netrc'.format(creds[1], homeDir + os.sep), shell=True)
        del homeDir
    del urs
    if aws:
        return(r.get('https://lpdaac.earthdata.nasa.gov/s3credentials').json())
    else:
        return('')

def build_xr(stac_dict):
    try:
        s30_stack = stackstac.stack(stac_dict['S30'], epsg=32613, resolution=30, assets=[i for i in lut['HLSS30'] if lut['HLSS30'][i] in needed_bands],
                                   chunksize=(4000, 4000)) #, reader=stackstac.reader_protocol.FakeReader
        s30_stack['band'] = [lut['HLSS30'][b] for b in s30_stack['band'].values]
        s30_stack['time'] = [datetime.fromtimestamp(t) for t in s30_stack.time.astype('int').values//1000000000]
        s30_stack = s30_stack.to_dataset(dim='band').reset_coords(['end_datetime', 'start_datetime'], drop=True)
    except ValueError:
        s30_stack = None
    try:
        l30_stack = stackstac.stack(stac_dict['L30'], epsg=32613, resolution=30, assets=[i for i in lut['HLSL30'] if lut['HLSL30'][i] in needed_bands],
                                   chunksize=(4000, 4000))
        l30_stack['band'] = [lut['HLSL30'][b] for b in l30_stack['band'].values]
        l30_stack['time'] = [datetime.fromtimestamp(t) for t in l30_stack.time.astype('int').values//1000000000]
        l30_stack = l30_stack.to_dataset(dim='band').reset_coords(['name', 'end_datetime', 'start_datetime'], drop=True)
    except ValueError:
        l30_stack = None
    if s30_stack is not None and l30_stack is not None:
        hls_stack = xr.concat([s30_stack, l30_stack], dim='time')
    elif s30_stack is not None:
        hls_stack = s30_stack
    elif l30_stack is not None:
        hls_stack = l30_stack
    else:
        print('No data found for date range')
    return hls_stack.chunk({'time': 1, 'y': -1, 'x': -1})
    
def get_hls(creds, hls_data={}, aws=False):
    #Seteup creds
    
    s3_cred = setup_netrc(creds,aws=aws)
    #define gdalenv
    if aws:
        
        env = dict(GDAL_DISABLE_READDIR_ON_OPEN='FALSE', 
                   #AWS_NO_SIGN_REQUEST='YES',
                   GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
                   GDAL_SWATH_SIZE='200000000',
                   VSI_CURL_CACHE_SIZE='200000000',
                   CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
                   GDAL_HTTP_UNSAFESSL='YES',
                   GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                   GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'),
                   AWS_REGION='us-west-2',
                   AWS_SECRET_ACCESS_KEY=s3_cred['secretAccessKey'],
                   AWS_ACCESS_KEY_ID=s3_cred['accessKeyId'],
                   AWS_SESSION_TOKEN=s3_cred['sessionToken'])
    else:
        env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
                   AWS_NO_SIGN_REQUEST='YES',
                   GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
                   GDAL_SWATH_SIZE='200000000',
                   VSI_CURL_CACHE_SIZE='200000000',
                   GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                   GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))


    os.environ.update(env)
    
    catalog = NASA_CMR_STAC(hls_data, aws)
    da  = build_xr(catalog)
    return da

In [ ]:
data_dict = {'date_range': [str(datetime(2021, 1, 1).date()), str(datetime.now().date())]}
tmp_data = get_hls(['spkearney', '1mrChamu'], hls_data=data_dict, aws=True)
bm_mod = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))
da = tmp_data.loc[dict(x=slice(517587.0, 527283.0), y=slice(4524402.0, 4514699.0))]
da_mask = mask_hls(da['FMASK'])
da = da.where(da_mask == 0)
da_bm = pred_bm(da, bm_mod)
da_bm = da_bm.where(da_bm > 0)
da_se = pred_bm_se(da, bm_mod)
da_se = da_se.where(da_bm > 0)
t0 = time.time()
print(da_bm.isel(time=1).values)
t1 = time.time()
print(t1-t0)

In [ ]:
cluster = LocalCluster(threads_per_worker=1)
cl = Client(cluster)
cl

In [ ]:
data_dict = {'date_range': [str(datetime(2021, 1, 15).date()), str(datetime.now().date())]}
tmp_data = get_hls(['spkearney', '1mrChamu'], hls_data=data_dict, aws=True)
bm_mod = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))
da = tmp_data.loc[dict(x=slice(517587.0, 527283.0), y=slice(4524402.0, 4514699.0))]
da_mask = mask_hls(da['FMASK'])
da = da.where(da_mask == 0)
da_bm = pred_bm(da, bm_mod)
da_bm = da_bm.where(da_bm > 0)
da_se = pred_bm_se(da, bm_mod)
da_se = da_se.where(da_bm > 0)
t0 = time.time()
print(da_bm.isel(time=1).values)
t1 = time.time()
print(t1-t0)

In [ ]:
from cartopy import crs
import geoviews as gv
import holoviews as hv
from copy import deepcopy
from holoviews import streams
import affine
from skimage.draw import polygon

cov_dict = {'R': 'Dry veg',
            'G': 'Green veg',
            'B': 'Bare ground'}

dat = da
dat['time'] = dat.time.dt.floor("D")
dat = dat.rename(dict(time='date'))
bm_mod = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))
cov_mod = pickle.load(open('src/models/CPER_HLS_to_LPI_cover_pls_binned_model.pk', 'rb'))

In [ ]:
class getData(pm.Parameterized):
    bm_mod = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))
    cov_mod = pickle.load(open('src/models/CPER_HLS_to_LPI_cover_pls_binned_model.pk', 'rb'))
    action = pm.Action(lambda x: x.param.trigger('action'), label='Load Data and Run Analysis')
    get_stats = pm.Action(lambda y: y.param.trigger('get_stats'), label='Calculate static stats')
    get_stats_ts = pm.Action(lambda y: y.param.trigger('get_stats_ts'), label='Calculate time-series stats')
    username_input = pn.widgets.PasswordInput(name='NASA Earthdata Login', placeholder='Enter Username...')
    password_input = pn.widgets.PasswordInput(name='', placeholder='Enter Password...')
    year_picker = pn.widgets.Select(name='Year', options=[2021], value=2021)
    thresh_picker = pn.widgets.IntSlider(name='Threshold', start=200, end=2000, step=25, value=500,
                                 format=PrintfTickFormatter(format='%d kg/ha'))
    
    data_dict = {'date_range': [str(datetime(year_picker.value, 1, 1).date()),
                                str(datetime(year_picker.value, 12, 31).date())]}
 
    chunks = {'date': 1, 'y': 250, 'x': 250}
    datCRS = crs.UTM(13)
    mapCRS = crs.GOOGLE_MERCATOR
    datProj = Proj(datCRS.proj4_init)
    mapProj = Proj(mapCRS.proj4_init)
    map_args = dict(crs=datCRS, rasterize=False, project=True, dynamic=True)
    base_opts = dict(projection=mapCRS, backend='bokeh', xticks=None, yticks=None, width=900, height=700,
                         padding=0, active_tools=['pan', 'wheel_zoom'], toolbar='left')
    map_opts = dict(projection=mapCRS, responsive=False, xticks=None, yticks=None, width=900, height=700,
                     padding=0, tools=['pan', 'wheel_zoom', 'box_zoom'],
                     active_tools=['pan', 'wheel_zoom'], toolbar='left')
    
    poly_opts = dict(fill_color=['', ''], fill_alpha=[0.0, 0.0], line_color=['#1b9e77', '#d95f02'],
                 line_width=[3, 3])    
    bg_col='#ffffff'

    css = '''
    .bk.box1 {
      background: #ffffff;
      border-radius: 5px;
      border: 1px black solid;
    }
    '''
    
    date = pn.widgets.DatePicker(name='Calendar', width=200)
      
    box_ctr_dat = [522435.0, 4519550.5]
    box_ctr_coord = datProj(box_ctr_dat[0], box_ctr_dat[1], inverse=True)
    box_ctr_map = mapProj(box_ctr_coord[0], box_ctr_coord[1], inverse=False)
    
    tiles = gv.tile_sources.EsriImagery.opts(**base_opts, level='glyph',
                                                                   xlim=(box_ctr_map[0] - 5000,
                                                                         box_ctr_map[0] + 5000),
                                                                   ylim=(box_ctr_map[1] - 5000,
                                                                         box_ctr_map[1] + 5000))
    labels = gv.tile_sources.EsriReference.opts(**base_opts, level='overlay')
       
    basemap = gv.tile_sources.EsriImagery.opts(projection=mapCRS, backend='bokeh', level='glyph')
    
    base_rng = hv.streams.RangeXY(source=tiles)
    
    polys = hv.Polygons([])

    poly_stream = streams.PolyDraw(source=polys, drag=True, num_objects=2,
                                    show_vertices=True, styles=poly_opts)
    edit_stream = streams.PolyEdit(source=polys, shared=True)
    
    gauge_obj = pn.indicators.Gauge(
        name='Biomass', bounds=(0, 2500), format='{value} kg/ha',
        colors=[(0.20, '#FF6E76'), (0.40, '#FDDD60'), (0.60, '#7CFFB2'), (1, '#58D9F9')],
        num_splits=5, value=0, align='center', title_size=12,
        start_angle=180, end_angle=0, height=200, width=300,
    )
    
    def __init__(self, **params):
        super(getData, self).__init__(**params)
        self.da = None
        self.da_sel = None
        self.da_cov = None
        self.da_bm = None
        self.da_bm_se = None
        self.da_thresh = None
        self.all_maps = None
        self.cov_map = None
        self.bm_map = None
        self.thresh_map = None
        
        self.cov_stats = None
        self.bm_stats = None
        self.thresh_stats = None
        self.stats_titles = None
                
        self.view = self._create_view()
        
    
    @pm.depends('action')
    def proc_data(self):
        message = 'Not yet launched'
        if self.username_input.value != '':
            message = 'username found'
            try:
                tmp_data = get_hls([self.username_input.value,self.password_input.value], 
                                   hls_data=self.data_dict, aws=True)
                message = self.base_rng.x_range
                coord_rng = self.mapProj(self.base_rng.x_range, self.base_rng.y_range, inverse=True)
                message = coord_rng
                dat_rng = np.round(self.datProj(coord_rng[0], coord_rng[1], inverse=False), 0)
                message = dat_rng
                self.da = tmp_data.loc[dict(x=slice(*dat_rng[0]), y=slice(*dat_rng[1][::-1]))]
                self.da['time'] = self.da.time.dt.floor("D")
                self.da = self.da.rename(dict(time='date'))
                self.da = self.da.chunk(self.chunks)
                da_mask = mask_hls(self.da['FMASK'])
                self.da = self.da.where(da_mask == 0)
                self.da = self.da.sel(date=self.da['eo:cloud_cover'] < 80)
                self.date.enabled_dates = [pd.Timestamp(x).to_pydatetime().date() for x in self.da['date'].values]
                if self.date.value is None:
                    self.da_sel = self.da.isel(date=-1).compute()
                    self.date.value = pd.to_datetime(self.da_sel.date.values).date()
                else:
                    date_idx = self.date.enabled_dates.index(self.date.value)
                    self.da_sel = self.da.isel(date=date_idx).compute()
                message = 'Success!' + str(datetime.now())
                return message
            except:
                return message + ': App Failure'
        else:
            return message
      
    @pm.depends('date.param')
    def create_maps(self):
        if self.da is not None and self.da_sel is not None:
            if pd.to_datetime(self.da_sel.date.values).date() != self.date.value:
                if self.date.value is not None:
                    date_idx = self.date.enabled_dates.index(self.date.value)
                    self.da_sel = self.da.isel(date=date_idx).compute()
            if self.edit_stream.data is not None:
                self.polys = self.edit_stream.element
                self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=2,
                                                    show_vertices=True, styles=self.poly_opts)
                self.edit_stream = streams.PolyEdit(source=self.polys, shared=True)
            elif self.poly_stream.data is not None:
                self.polys = self.poly_stream.element
                self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=2,
                               show_vertices=True, styles=self.poly_opts)
                self.edit_stream = streams.PolyEdit(source=self.polys, shared=True)
            else:
                self.polys = self.polys
            da_bm = self.da_sel.map_blocks(pred_bm, template=self.da_sel['BLUE'],
                                     kwargs=dict(model=self.bm_mod))
            da_bm = da_bm.where(da_bm > 0)
            da_bm.name = 'Biomass'
            
            da_cov_temp = self.da_sel[['BLUE', 'GREEN', 'RED', 'NIR1']].rename(dict(BLUE='SD', RED='BARE', NIR1='LITT'))
            da_cov = self.da_sel.map_blocks(pred_cov, template=da_cov_temp,
                         kwargs=dict(model=self.cov_mod))
            da_cov = da_cov[['SD', 'GREEN', 'BARE']].to_array(dim='type')
            da_cov = da_cov.where((da_cov < 1.0) | (da_cov.isnull()), 1.0)
            da_cov.name = 'Cover'
            
            da_bm_se = self.da_sel.map_blocks(pred_bm_se, template=self.da_sel['BLUE'],
                                         kwargs=dict(model=self.bm_mod))
           
            da_thresh_pre = (np.log(self.thresh_picker.value) - xr.ufuncs.log(da_bm)) / da_bm_se
            da_thresh = da_thresh_pre.map_blocks(xr_cdf, template=self.da_sel['BLUE'])
            #da_thresh = pred_bm_thresh(da_bm, da_bm_se, self.thresh_picker.value)
            da_thresh.name = 'Threshold'
            
            da_ndvi = ndvi_func(self.da_sel)
            
            self.cov_map = da_cov.hvplot.rgb(x='x', y='y', bands='type',
                                        **self.map_args).opts(**self.map_opts)
            self.bm_map = da_bm.hvplot(x='x', y='y',
                                  cmap='Inferno', clim=(100, 1000), colorbar=False,
                                  **self.map_args).opts(**self.map_opts)
            self.thresh_map = da_thresh.hvplot(x='x', y='y',
                                          cmap='YlOrRd', clim=(0.05, 0.95), colorbar=False,
                                          **self.map_args).opts(**self.map_opts)
            
            self.ndvi_map = da_ndvi.hvplot(x='x', y='y',
                              cmap='Viridis', clim=(0.05, 0.50), colorbar=False,
                              **self.map_args).opts(**self.map_opts)

            self.da_cov = da_cov
            self.da_bm = da_bm
            self.da_bm_se = da_bm_se
            self.da_thresh = da_thresh
            self.da_ndvi = da_ndvi
            
            self.all_maps = pn.Tabs(
                ('Cover', self.basemap * self.cov_map * self.polys),
                ('Biomass', self.basemap * self.bm_map * self.polys),
                ('Biomass threshold', self.basemap * self.thresh_map * self.polys), 
                ('Greenness (NDVI)', self.basemap * self.ndvi_map * self.polys))
            
            #self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=2,
            #                        show_vertices=True, styles=self.poly_opts)
        
            return self.all_maps
        else:
            return pn.Column(self.tiles * self.labels)
        
    @pm.depends('get_stats')
    def show_stats(self):
        if self.poly_stream.data is not None:
            markdown_list = []
            thresh_list = []
            bm_list = []
            cov_list = []
            stats_rows = []
            for idx, ps_c in enumerate(self.poly_stream.data['line_color']):
                r, c = polygon(self.poly_stream.data['xs'][idx]*-1, self.poly_stream.data['ys'][idx])
                r = r * -1.0
                
                da_cov_tmp = self.cov_map[r][:,c].data
                cov_factors = [k for k in app.cov_map.data.keys() if k not in ['y', 'x']]
                cov_labels = [cov_dict[i] for i in cov_factors]
                cov_labels.append('Litter')
                cov_vals = [round(float(da_cov_tmp[f].mean()), 2) for f in cov_factors]
                cov_vals.append(round(1.0 - np.sum(cov_vals), 2))
                pct_fmt = NumeralTickFormatter(format="0%")
                cov_colors = hv.Cycle(['red', 'green', 'blue', 'orange'])
                cov_scatter_tmp = hv.Overlay([hv.Scatter(f) for f in list(zip(cov_labels, cov_vals))]) \
                    .options({'Scatter': dict(xformatter=pct_fmt,
                                              size=15,
                                              fill_color=cov_colors,
                                              line_color=cov_colors,
                                              ylim=(0, 1))})
                cov_spike_tmp = hv.Overlay([hv.Spikes(f) for f in cov_scatter_tmp])\
                    .options({'Spikes': dict(color=cov_colors, line_width=4,
                                             labelled=[], invert_axes=True, color_index=None,
                                             ylim=(0, 1))})
                cov_list.append(pn.Column((cov_spike_tmp * cov_scatter_tmp).options(height=200,
                                                                                    width=300,
                                                                                    bgcolor=self.bg_col,
                                                                                    toolbar=None),
                                          css_classes=['box1'], margin=5, align='center'))
                bm_dat_tmp = self.bm_map[r][:,c].data
                bm_hist_tmp = bm_dat_tmp.hvplot.hist('Biomass', xlim=(0, 2000),
                                                    bins=np.arange(0, 10000, 20))\
                    .opts(height=200, width=300, fill_color=ps_c, fill_alpha=0.6,
                          line_color='black', line_width=0.5, line_alpha=0.6,
                          bgcolor=self.bg_col, align='center').options(toolbar=None)
                bm_gauge_obj = deepcopy(self.gauge_obj)
                bm_gauge_obj.value = int(bm_dat_tmp.mean()['Biomass'])
                bm_list.append(pn.Column(bm_gauge_obj,
                                         css_classes=['box1'], margin=5, align='center'))
                markdown = pn.pane.Markdown('# Region ' + str(idx+1) + ' :', align='center',
                                            style={'font-family': "serif",
                                                   'color': ps_c})
                markdown_list.append(pn.Column(markdown, css_classes=['box1'], margin=5, align='center'))
                thresh_pct = round(float(bm_dat_tmp.where(bm_dat_tmp < 500).count()['Biomass'])/
                                   float(bm_dat_tmp.count()['Biomass']) * 100, 0)
                thresh_text = pn.pane.Markdown(f'**{thresh_pct}%** of the region is estimated to have biomass ' +
                                               f'less than {500} kg/ha.',
                                               style={'font-family': "Helvetica"})
                thresh_list.append(pn.Column(bm_hist_tmp * hv.VLine(x=500).opts(line_color='black', height=200),
                                             thresh_text,
                                             css_classes=['box1'], margin=5, align='center'))
                stats_rows.append(pn.Row(markdown_list[idx], bm_list[idx], thresh_list[idx], cov_list[idx]))
            self.stats_titles = pn.Column(*markdown_list)
            self.cov_stats = pn.Column(*cov_list)
            self.bm_stats = pn.Column(*bm_list)
            self.thresh_stats = pn.Column(*thresh_list)
            return pn.Column(*stats_rows)
        else:
            return pn.Row(None)
    
    def _create_view(self):
        layout = pn.Column(pn.Row(pn.Column(self.username_input,
                                  self.password_input,
                                  pn.Param(self.param,
                                           widgets={'action': pn.widgets.Button(name='Get data', width=200),
                                                    'get_stats': pn.widgets.Button(name='Calculate static stats', width=200),
                                                   'get_stats_ts': pn.widgets.Button(name='Calculate time-series stats', width=200)},
                                  show_name=False),
                         self.proc_data,
                         self.date), pn.Column(self.create_maps)),
                                               self.show_stats)
        return layout
                                        

In [ ]:
app = getData()
app.view.servable()

In [ ]:
from src.hls_funcs.masks import shp2mask
polymask_list = []
for idx, ps_c in enumerate(app.poly_stream.data['line_color']):
    r, c = polygon(app.poly_stream.data['xs'][idx]*-1, app.poly_stream.data['ys'][idx])
    r = r * -1.0
    r_map, c_map = app.mapProj(r, c, inverse=True)
    r_dat, c_dat = app.datProj(r_map, c_map, inverse=False)
    geometries = {
        "type": "Polygon",
        "coordinates": [list(map(list, zip(r_dat, c_dat)))]
    }
    ta = affine.Affine(30.0, 0.0, float(app.da['x'].min()),
                       0.0, -30.0, float(app.da['y'].max()))
    polymask_tmp = shp2mask([geometries], app.da,
                         transform=ta, outshape=app.da['BLUE'].isel(date=0).shape, default_value=1)
    polymask_tmp['region'] = idx + 1
    polymask_tmp['line_color'] = ps_c
    polymask_list.append(polymask_tmp)
polymask = xr.concat(polymask_list, dim='region')
polymask
# don't want to fetch data twice, so need to get all masks at the same time with labels, then mean over date+label

In [ ]:
ndvi_ts_tmp = ndvi_func(app.da.sel(date=app.da['eo:cloud_cover'] < 80).where(poly_mask == 1)).mean(dim=['y', 'x'])
ndvi_ts_tmp.interp().plot()